In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import torchvision
import torchvision.transforms as transforms

from collections import namedtuple
from itertools import product

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from collections import OrderedDict

from HWCRUtils import HWCRUtils
from Train import Train_Manager
from Test import Test_Manager
from CNN import Network


from collections import OrderedDict

In [4]:
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

data_train = np.load('train_data.pkl', allow_pickle=True)
print("data_train")
print(data_train.shape)
nd_arr = np.array(data_train[0])
print(nd_arr.shape)

labels = np.load('finalLabelsTrain.npy', allow_pickle=False)
print(labels.shape)
print(labels[100])

data_train
(6400,)
(50, 49)
(6400,)
3.0


In [3]:


class HandWrittenRecognitionDeep:
    def __init__(self):
        self.__train_set = None
        self.__test_set = None
        self.__validation_set = None
        self.__test_set_size = None
        self.__validation_set_size = None

    def perform_hand_written_char_recognition(self, data_set_path, label_set_path, image_dims, parameters, split_size,
                                              epochs=10):
        self.__split_train_test_validation_set(data_set_path, label_set_path, image_dims, split_size)
        network = self.__train_model(parameters, epochs)
        return network
        
    def test_hand_written_char_recognition(self, parameters, network):
         self.__test_model(network, parameters)  

    def __split_train_test_validation_set(self, data_set_path, label_set_path, image_dims, split_size):
        train_data_set, labels_set = HWCRUtils.read_dataset(data_set_path, label_set_path, image_dims)
        X_train, X_test, Y_train, Y_test = HWCRUtils.spilt_data_set(train_data_set,
                                                                    labels_set,
                                                                    split_size=split_size)
        X_train, X_val, Y_train, Y_val = HWCRUtils.spilt_data_set(X_train,
                                                                  Y_train,
                                                                  split_size=split_size)
        train_set = HWCRUtils.convert_to_tensor(X_train, Y_train)
        test_set = HWCRUtils.convert_to_tensor(X_test, Y_test)
        val_set = HWCRUtils.convert_to_tensor(X_val, Y_val)

        self.__train_set = train_set
        self.__test_set = test_set
        self.__validation_set = val_set
        self.__test_set_size = Y_test.shape[0]
        self.__validation_set_size = Y_test.shape[0]

    def __train_model(self, parameters, epochs=10):
        train = Train_Manager()
        network = Network()
        response = train.train_data_set(self.__train_set, network,
                                        HWCRUtils.get_runs(parameters)[0],
                                        epochs)
        return response["network"]

    def __test_model(self, network, parameters):
        test = Test_Manager()
        ret = test.test_data_set(self.__validation_set, network, HWCRUtils.get_runs(parameters)[0])
        percnt_correct = (ret['total_correct'] / self.__test_set_size) * 100
        print(f"total loss test: {ret['total_loss']}")
        print(f"correctly predicted: {ret['total_correct']}")
        print(f"actual correct: {self.__test_set_size}")
        print(f"% correct: {percnt_correct}")

In [4]:
data_set_path = "train_data.pkl"
label_set_path = "finalLabelsTrain.npy"
image_dims = (52, 52)
epochs = 18
split_size = 0.01
parameters = OrderedDict(
    lr=[0.01],
    batch_size=[100]
)

hwRD = HandWrittenRecognitionDeep()
network = hwRD.perform_hand_written_char_recognition(data_set_path, label_set_path, image_dims, parameters, split_size,
                                               epochs)
#     hwRD.test_hand_written_char_recognition(parameters, network,
#                                               epochs=10)

Reading data from pkl
training starts now
epoch: 0, total_correct: 4190, actual_correct: 6272, % correct: 66.80484693877551, loss: 75.91474455595016
epoch: 1, total_correct: 5433, actual_correct: 6272, % correct: 86.62308673469387, loss: 24.95569597184658
epoch: 2, total_correct: 5777, actual_correct: 6272, % correct: 92.1077806122449, loss: 14.833492986857891
epoch: 3, total_correct: 5936, actual_correct: 6272, % correct: 94.64285714285714, loss: 10.268797885626554
epoch: 4, total_correct: 5986, actual_correct: 6272, % correct: 95.44005102040816, loss: 8.42610185034573
epoch: 5, total_correct: 6013, actual_correct: 6272, % correct: 95.87053571428571, loss: 7.370262671262026
epoch: 6, total_correct: 6041, actual_correct: 6272, % correct: 96.31696428571429, loss: 6.739381009712815
epoch: 7, total_correct: 6102, actual_correct: 6272, % correct: 97.28954081632652, loss: 5.1121900994330645
epoch: 8, total_correct: 6145, actual_correct: 6272, % correct: 97.9751275510204, loss: 3.62964907474

In [25]:
hwRD.test_hand_written_char_recognition(parameters, network)

total loss test: 0.3359124958515167
correctly predicted: 55
actual correct: 64
% correct: 85.9375


In [22]:
network

Network(
  (conv1): Conv2d(1, 128, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=512, out_features=1024, bias=True)
  (out): Linear(in_features=1024, out_features=9, bias=True)
)